<a href="https://colab.research.google.com/github/myracheng/155kaggle1/blob/master/Copy_of_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout,Activation,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import SGD
from sklearn.svm import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, learning_curve, validation_curve, cross_val_score

from sklearn.ensemble import GradientBoostingClassifier,VotingClassifier,RandomForestClassifier,AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import  roc_auc_score
from sklearn.ensemble import VotingClassifier
import xgboost as xgb

import pandas as pd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [0]:
def add_fields(df, with_y):
  df = df.fillna(0)
  df2 = pd.DataFrame()

  df2['avg_bid'] = (df['bid1'] * df['bid1vol'] + df['bid2'] * df['bid2vol'] + df['bid3'] * df['bid3vol'] + df['bid4'] * df['bid4vol'] + df['bid5'] * df['bid5vol']) / (df['bid1vol'] + df['bid2vol'] + df['bid3vol'] + df['bid4vol'] + df['bid5vol'])
  df2['avg_ask'] = (df['ask1'] * df['ask1vol'] + df['ask2'] * df['ask2vol'] + df['ask3'] * df['ask3vol'] + df['ask4'] * df['ask4vol'] + df['ask5'] * df['ask5vol']) / (df['ask1vol'] + df['ask2vol'] + df['ask3vol'] + df['ask4vol'] + df['ask5vol'])

  df2['mid_diff'] = df['last_price'] - df['mid']

  df2['diff_bid1'] = df['last_price'] - df['bid1']
  df2['diff_bid2'] = df['last_price'] - df['bid2']
  df2['diff_bid3'] = df['last_price'] - df['bid3']
  df2['diff_bid4'] = df['last_price'] - df['bid4']
  df2['diff_bid5'] = df['last_price'] - df['bid5']

  df2['diff_mid_ask1'] = df['mid'] - df['ask1']
  df2['diff_mid_ask2'] = df['mid'] - df['ask2']
  df2['diff_mid_ask3'] = df['mid'] - df['ask3']
  df2['diff_mid_ask4'] = df['mid'] - df['ask4']
  df2['diff_mid_ask5'] = df['mid'] - df['ask5']

  df2['avg_bid_last_price_diff'] = df2['avg_bid'] - df['last_price']
  df2['avg_ask_last_price_diff'] = df2['avg_ask'] - df['last_price']
  df2['avg_bid_mid_diff'] = df2['avg_bid'] - df['mid']
  df2['avg_ask_mid_diff'] = df2['avg_ask'] - df['mid']

  df2['best_offer_vol_diff'] = df['bid1vol'] - df['ask1vol']
  df2['avg_best_offer'] = (df['bid1'] * df['bid1vol'] + df['ask1'] * df['ask1vol']) / (df['bid1vol'] + df['ask1vol'])
  df2['mid_avg_best_offer_diff'] = df['mid'] - df2['avg_best_offer']
  df2['last_price_avg_best_offer_diff'] = df['last_price'] - df2['avg_best_offer']
  df2['d_open_interest'] = df['d_open_interest'] 
  df2['transacted_qty'] = df['transacted_qty'] 

  #new features
  df2['mids_mult'] = df2['diff_mid_ask1']*df2['diff_mid_ask2']*df2['diff_mid_ask3']*df2['diff_mid_ask4']*df2['diff_mid_ask5']
  df2['last_price_mult'] = df2['diff_bid1']*df2['diff_bid2']*df2['diff_bid3']*df2['diff_bid4']*df2['diff_bid5']

  if with_y == True:
    y = df['y']
  else:
    y = []

    
  X = df2.loc[:, df2.columns != 'y']

  
  return X, y

In [0]:
from zipfile import ZipFile 
file_name = "drive/My Drive/train.csv.zip"
  
# # opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipF: 
    # printing all the contents of the zip file 
    zipF.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    data = zipF.extract('train.csv') 
    print('Done!')

file_name = "drive/My Drive/test.csv.zip"
  
# # opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipF: 
    # printing all the contents of the zip file 
    zipF.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    data = zipF.extract('test.csv') 
    print('Done!')

File Name                                             Modified             Size
train.csv                                      2020-02-09 10:27:26     77009507
Extracting all the files now...
Done!
File Name                                             Modified             Size
test.csv                                       2020-02-09 10:27:26     24378147
Extracting all the files now...
Done!


# Create Training Set


In [0]:
train_df = pd.read_csv('train.csv', index_col=0)
train_df, y = add_fields(train_df, with_y=True)

X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.2, random_state=0)


In [0]:
from sklearn.naive_bayes import GaussianNB

# Train Ensemble Model


In [0]:
clf1 = GradientBoostingClassifier()
clf2 = RandomForestClassifier()
clf4 = AdaBoostClassifier()
clf5 = LGBMClassifier()
clf6 = XGBClassifier()
eclf1 = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2), ('ada', clf4),('lgbm', clf5),('xgb', clf6)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
print(eclf1.score(X_test, y_test))

0.6685995475876971


In [0]:
y_probs = eclf1.predict_proba(X_test)
print(roc_auc_score(y_test, y_probs[:,1]))

0.6769768234454128


In [0]:
test_df = pd.read_csv('test.csv', index_col=0)
test_df, __ = add_fields(test_df, with_y=False)

probbs = eclf1.predict_proba(test_df)
test_df['Predicted'] = probbs[:, 1]

header = ["id", "Predicted"]
test_df['id']=test_df.index
test_df.to_csv('output_withmults.csv', columns = header,index=False)




In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
import numpy as np
np.arange(0.01,0.3,0.05)

array([0.01, 0.06, 0.11, 0.16, 0.21, 0.26])

In [0]:
for eta in np.arange(0.01,0.3,0.1):
  print(eta)
  clf1 = GradientBoostingClassifier()
  clf2 = RandomForestClassifier()
  clf4 = AdaBoostClassifier()
  clf5 = LGBMClassifier()
  clf6 = XGBClassifier(learning_rate=eta)

  eclf1 = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2),('c',clf4),('lgbm', clf5),('xgb', clf6)], voting='soft')
  eclf1 = eclf1.fit(X_train, y_train)
  print(eclf1.score(X_test, y_test))
  y_probs = eclf1.predict_proba(X_test)
  print(roc_auc_score(y_test, y_probs[:,1]))

0.01
0.6689371687092744
0.6774258054493661
0.11
0.6685742260035787
0.6768692529481757
0.21000000000000002
0.6687514770924069
0.6771527766440537


In [0]:
# for eta in np.arange(3,11,2):
print(eta)
clf1 = GradientBoostingClassifier()
clf2 = RandomForestClassifier()
# clf3 = MultinomialNB()
clf4 = AdaBoostClassifier()
clf5 = LGBMClassifier()
clf6 = XGBClassifier(max_depth=7,learning_rate=0.2)

eclf1 = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2),('c',clf4),('lgbm', clf5),('xgb', clf6)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
print(eclf1.score(X_test, y_test))
y_probs = eclf1.predict_proba(X_test)
print(roc_auc_score(y_test, y_probs[:,1]))

3
0.6690384550457477
0.6768488150729045
5
0.6689456092373138
0.6775431648750256
7
0.6692832303588913
0.6780906340999644
9
0.6695786488402714
0.6779608793429491


In [0]:
# for eta in np.arange(3,11,2):
print(eta)
clf1 = GradientBoostingClassifier()
clf2 = RandomForestClassifier()
# clf3 = MultinomialNB()
clf4 = AdaBoostClassifier()
clf5 = LGBMClassifier()
clf6 = XGBClassifier(max_depth=7,learning_rate=0.2)

eclf1 = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2),('c',clf4),('lgbm', clf5),('xgb', clf6)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
print(eclf1.score(X_test, y_test))
y_probs = eclf1.predict_proba(X_test)
print(roc_auc_score(y_test, y_probs[:,1]))

9
0.6697221378169418
0.678228210235587


In [0]:
test_df = pd.read_csv('test.csv', index_col=0)
test_df, __ = add_fields(test_df, with_y=False)

probbs = eclf1.predict_proba(test_df)
test_df['Predicted'] = probbs[:, 1]

header = ["id", "Predicted"]
test_df['id']=test_df.index
test_df.to_csv('output_withmults_bestparams.csv', columns = header,index=False)




# Create .csv of Test Set

